In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, mean_absolute_error, mean_squared_error, r2_score
import torch
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [6]:
path = "../IDMT-Traffic/datasets/df_main.csv"
data_uncleaned = pd.read_csv(path)

/var/folders/xt/qp4h10d92h76_sfj0rh4m3v00000gn/T/ipykernel_18075/2247780661.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data_uncleaned = pd.read_csv(path)


In [7]:
path = "../IDMT-Traffic/datasets/df_main.csv"
data = pd.read_csv(path) 

data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.drop(["sample_pos"], axis=1, inplace=True)  
data["speed_kmh"] = data["speed_kmh"].astype("str")
data.dropna(inplace=True)  

# Verify no NaN values remain
print(data.isnull().values.any()) 

/var/folders/xt/qp4h10d92h76_sfj0rh4m3v00000gn/T/ipykernel_18075/3845355249.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


False


Encoding + Mapping for the encoded values 

In [4]:
le = LabelEncoder()

for col in data.columns:
    if (data[col].dtype == "object" or data[col].dtype == "category" or data[col].dtype == "bool") and not (col == "file"):
        #encode column and save in new column
        encoded_col = le.fit_transform(data[col])
        # add _encoded to the original column name
        new_col_name = col + "_encoded"
        # insert encoded column after the original one 
        col_index = data.columns.get_loc(col) + 1
        data.insert(col_index, new_col_name, encoded_col)

        mapping = {category: idx for idx, category in enumerate(le.classes_)}
        print(f"Mapping for column '{col}': {mapping}")

Mapping for column 'is_background': {False: 0}
Mapping for column 'date_time': {'2019-10-22-08-40': 0, '2019-10-22-15-30': 1, '2019-10-22-16-35': 2, '2019-10-23-08-35': 3, '2019-10-23-16-20': 4, '2019-11-12-09-00': 5, '2019-11-12-10-00': 6, '2019-11-12-16-00': 7, '2019-11-13-08-00': 8, '2019-11-13-15-30': 9, '2019-11-13-15-55': 10, '2019-11-18-07-25': 11, '2019-11-18-15-45': 12, '2019-11-18-16-40': 13, '2019-11-19-07-25': 14, '2019-11-19-08-30': 15, '2019-11-19-15-25': 16, '2019-11-19-16-30': 17, '2020-08-29-10-02': 18, '2020-08-29-11-02': 19, '2020-08-29-12-07': 20, '2020-08-29-14-02': 21, '2020-08-29-16-07': 22}
Mapping for column 'location': {'Fraunhofer-IDMT': 0, 'Hohenwarte': 1, 'Langewiesener-Strasse': 2, 'Schleusinger-Allee': 3}
Mapping for column 'speed_kmh': {'30': 0, '50': 1, '70': 2, 'UNK': 3}
Mapping for column 'daytime': {'A': 0, 'M': 1}
Mapping for column 'weather': {'D': 0, 'W': 1}
Mapping for column 'vehicle': {'B': 0, 'C': 1, 'M': 2, 'T': 3}
Mapping for column 'source_

In [8]:
data.to_csv("../IDMT-Traffic/datasets/df_main_encoded.csv")

In [9]:
# Find all columns that have "_encoded" versions
encoded_columns = [col for col in data.columns if '_encoded' in col]

# Derive the base names of the encoded columns
base_columns_to_drop = [col.replace('_encoded', '') for col in encoded_columns]

# Drop the base columns that have "_encoded" counterparts
data = data.drop(columns=base_columns_to_drop)
data.to_csv("../IDMT-Traffic/datasets/df_main_encoded_only.csv", index=False)

In [10]:
encoded = pd.read_csv("../IDMT-Traffic/datasets/df_main_encoded.csv")
encoded_only = pd.read_csv("../IDMT-Traffic/datasets/df_main_encoded_only.csv")